In [0]:
from pyspark.sql.functions import *

# Read the icd_codes file from bronze
icd_codes_df = spark.read.parquet("abfss://bronze@databricksdevetl.dfs.core.windows.net/icd_codes/")

icd_codes_df.createOrReplaceTempView("icd_codes")
display(icd_codes_df)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_dev.silver.icd_codes (
    icd_code STRING,
    icd_code_type STRING,
    code_description STRING,
    inserted_date DATE,
    updated_date DATE,
    is_current_flag BOOLEAN
)

In [0]:
%sql
MERGE INTO
  databricks_dev.silver.icd_codes AS target
USING
  icd_codes AS source
ON target.icd_code = source.icd_code
WHEN MATCHED AND
  target.code_description != source.code_description
  THEN UPDATE SET
  target.code_description = source.code_description,
  target.updated_date = source.updated_date,
  target.is_current_flag = False
WHEN NOT MATCHED THEN INSERT (
    icd_code, icd_code_type, code_description, inserted_date, updated_date, is_current_flag
  )
  VALUES (
    source.icd_code,
    source.icd_code_type,
    source.code_description,
    source.inserted_date,
    source.updated_date,
    source.is_current_flag
  )

In [0]:
%sql
SELECT * FROM databricks_dev.silver.icd_codes;